In [ ]:
!pip install autokeras

     |████████████████████████████████| 174kB 14.7MB/s 
     |████████████████████████████████| 71kB 8.4MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=c8d285773a1ab397e0cbcd24f96b7b6f2ebc32356a5bfca5e8cd1c712213f357
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=743d0dee34772c1cb5ed0a591b165ee590c3f70bc971e2a0fd9e3bda57f0fa18
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist

import autokeras as ak

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape)  # (60000, 28, 28)
print(y_train.shape)  # (60000,)
print(y_train[:3])  # array([7, 2, 1], dtype=uint8)

11493376/11490434 [==============================] - 0s 0us/step
(60000, 28, 28)
(60000,)
[5 0 4]


In [ ]:
# Initialize the image classifier.
clf = ak.ImageClassifier(overwrite=True, max_trials=1)
# Feed the image classifier with training data.
clf.fit(x_train, y_train, epochs=10)


# Predict with the best model.
predicted_y = clf.predict(x_test)
print(predicted_y)


# Evaluate the best model with testing data.
print(clf.evaluate(x_test, y_test))

Trial 1 Complete [00h 21m 19s]
val_loss: 0.04003217816352844

Best val_loss So Far: 0.04003217816352844
Total elapsed time: 00h 21m 19s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
1875/1875 [==============================] - 152s 81ms/step - loss: 0.2911 - accuracy: 0.9108
Epoch 2/10
1875/1875 [==============================] - 151s 81ms/step - loss: 0.0795 - accuracy: 0.9761
Epoch 3/10
1875/1875 [==============================] - 151s 81ms/step - loss: 0.0619 - accuracy: 0.9809
Epoch 4/10
1875/1875 [==============================] - 151s 81ms/step - loss: 0.0509 - accuracy: 0.9836
Epoch 5/10
1875/1875 [==============================] - 151s 81ms/step - loss: 0.0469 - accuracy: 0.9856
Epoch 6/10
1875/1875 [==============================] - 151s 81ms/step - loss: 0.0424 - accuracy: 0.9862
Epoch 7/10
1875/1875 [==============================] - 151s 81ms/step - loss: 0.0399 - accuracy: 0.9873
Epoch 8/10
1875/1875 [==============================] - 150s 80ms/step - loss: 0.0339 - acc

DIY automl based on sklearn model

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer, make_column_selector

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, OneHotEncoder
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC,SVC
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import balanced_accuracy_score

notes:

*   The purpose of the pipeline is to assemble several steps that can be cross-validated together while setting different parameters.

*   sklearn.pipeline.Pipeline(steps, *, memory=None, verbose=False);steps=[List of (name, transform) tuples (implementing fit/transform) that are chained]

*   ColumnTransformer: Applies transformers to columns of an array or pandas DataFrame.
*   RandomizedSearchCV(estimator, param_distributions, *, n_iter=10, scoring=None, n_jobs=None, refit=True, cv=None, verbose=0, pre_dispatch='2*n_jobs', random_state=None, error_score=nan, return_train_score=False): param_distributions = optimization_grid = dict or list of dicts




In [ ]:
class AutoMLClassifier:
  def __init__(self, scoring_fuction = 'balanced_accuracy', n_iter = 50):
    self.scoring_fuction = scoring_function
    self.n_iter = n_iter
  
  def fit(self, X, y):
    X_train = X
    y_train = y

    # detect distinct value of categorical variables
    categorical_values = []
    #cat_subset = X_train.select_dtypes(include = ['object','category','bool'])#e.g. for pd_dataframe
    cat_subset = X_train['label']#change in terms of labeling in image dataset

    for i in range(cat_su bset.shape[1]):
      categorical_values.append(list(cat_subset.iloc[:,i].dropna().unique()))

    # defining a pre-processing pipeline
    num_pipeline = Pipeline([
      ('cleaner',SimpleImputer()),
      ('scaler',StandardScaler())
    ])

    cat_pipeline = Pipeline([
      ('cleaner',SimpleImputer(strategy = 'most_frequent')),
      ('encoder',OneHotEncoder(sparse = False, categories=categorical_values))
    ])


    preprocessor = ColumnTransformer([
      ('numerical', num_pipeline, make_column_selector(dtype_exclude=['object','category','bool'])),
      ('categorical', cat_pipeline, make_column_selector(dtype_include=['object','category','bool']))
    ])#need to be transfered for image data

    model_pipeline_steps = []
    model_pipeline_steps.append(('preprocessor',preprocessor))
    model_pipeline_steps.append(('feature_selector',SelectKBest(f_classif,k='all'))) # “all” option bypasses selection, for use in a parameter search.
    model_pipeline_steps.append(('estimator',LogisticRegression()))
    model_pipeline = Pipeline(model_pipeline_steps)

    total_features = preprocessor.fit_transform(X_train).shape[1]

    # set randam search optimizers
    optimization_grid = []
    # KNN
    optimization_grid.append({
        'preprocessor_numerical_scaler':[RobustScaler(),StandardScaler(),MinMaxScalar()],
        'preprocessor_numerical_cleaner_strategy':['mean','median'],
        'feature_selector_k':list(np.arange(1,total_features,5))+['all'],
        'estimator':[KNeighborsClassifier()],
        'estimator__weights':['uniform','distance'],
        'estimator__n_neighbors':np.arange(1,20,1)
    })

    # Random forest
    optimization_grid.append({
        'preprocessor__numerical__scaler':[None],
        'preprocessor__numerical__cleaner__strategy':['mean','median'],
        'feature_selector__k': list(np.arange(1,total_features,5)) + ['all'],
        'estimator':[RandomForestClassifier(random_state=0)],
        'estimator__n_estimators':np.arange(5,500,10),
        'estimator__criterion':['gini','entropy']
    })

    # Gradient boosting
    optimization_grid.append({
        'preprocessor__numerical__scaler':[None],
        'preprocessor__numerical__cleaner__strategy':['mean','median'],
        'feature_selector__k': list(np.arange(1,total_features,5)) + ['all'],
        'estimator':[GradientBoostingClassifier(random_state=0)],
        'estimator__n_estimators':np.arange(5,500,10),
        'estimator__learning_rate':np.linspace(0.1,0.9,20),
    })

    # Linear SVM
    optimization_grid.append({
        'preprocessor__numerical__scaler':[RobustScaler(),StandardScaler(),MinMaxScaler()],
        'preprocessor__numerical__cleaner__strategy':['mean','median'],
        'feature_selector__k': list(np.arange(1,total_features,5)) + ['all'],
        'estimator':[LinearSVC(random_state = 0)],
        'estimator__C': np.arange(0.1,1,0.1),
        
    }) 
    
    # Logistic regression
    optimization_grid.append({
        'preprocessor__numerical__scaler':[RobustScaler(),StandardScaler(),MinMaxScaler()],
        'preprocessor__numerical__cleaner__strategy':['mean','median'],
        'feature_selector__k': list(np.arange(1,total_features,5)) + ['all'],
        'estimator':[LogisticRegression()]
    })

    # operate randam search
    search = RandomizedSearchCV(
      model_pipeline,
      optimization_grid,
      n_iter=self.n_iter,
      scoring = self.scoring_function, 
      n_jobs = -1, 
      random_state = 0, 
      verbose = 3,
      cv = 5 #5-fold cross validation
    )
    search.fit(X_train.y_train)
    self.best_estimator = search.best_estimator_
    self.best_pipeline = search.best_params_

  def predict(self,X,y=None):
    return self.best_estimator.predict(X)

  def predict_prob(self,X,y=None):
    return self.best_estimator.predict_proba(X)

data load

In [ ]:
d = load_data()
#labeling
y = d['label']# adjust for needed
X = data_input_transform# adjust for needed
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=10)

model = AutoMLClassifier()
model.fit(X_train,y_train)

The balanced accuracy in binary and multiclass classification problems to deal with imbalanced datasets. It is defined as the average of recall obtained on each class

In [ ]:
balanced_accuracy_score(y_test, model.predict(X_test))

fit model, see which model and preprocessing parameters have been chosen

In [ ]:
model.best_pipeline